In [1]:
import syft as sy
import torch as th

hook = sy.TorchHook(th)

# Step one: defining the parties

Alice and bob will be the data owners.
There is an implicit third party called "me" which is the party which runs the code, holds the pointers to the data, and will also be the crypto provider because we haven't defined a separate crypto provider.

In [2]:
alice = sy.VirtualWorker(hook, id='alice')
bob = sy.VirtualWorker(hook, id='bob')

alice.add_workers([bob])
bob.add_workers([alice])

Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:bob #objects:0>

# Step two: storing the data
Two tensors _x_ and _y_ are defined and sent to Alice and Bob. 

In [3]:
x = th.Tensor([1,2,3])
y = th.Tensor([4,5,6])

x_p = x.send(alice)
y_p = y.send(bob)

alice._objects

{93107390310: tensor([1., 2., 3.])}

# Step three: sharing secrets
`x` and `y` are converted to fixed precision values and encrypted into shared secrets.
Alice and Bob now contain an AdditiveSharingTensor, which contains pointers to all the secret shares at the different locations.

In [4]:
x_share = x_p.fix_prec().share(bob, alice)
y_share = y_p.fix_prec().share(bob, alice)

alice._objects

{93107390310: tensor([1., 2., 3.]),
 14474660578: tensor([ 3509559978586529153,  2176485925326727821, -3547211131702888029]),
 34608355359: (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:29603345025 -> bob:68716225855]
 	-> [PointerTensor | me:4526064845 -> alice:14474660578]
 	*crypto provider: me*,
 22089987977: tensor([-4615140013289734418, -2309704696375277563,  6548228024791879498])}

# Step four: collecting the pointers for the shared secrets and performing the analysis
In order to perform computations on the AdditiveSharingTensors they need to be moved to the same party by calling `get`. Mind you, this does not move the secret values, merely the pointers to the cryptographical pieces that are located at the different parties.

After moving the tensors we can perform the analysis. The result of the analysis will be another AdditiveSharingTensor.

In [5]:
secret_result = (x_share.get() + y_share.get())/2
secret_result

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:16437238094 -> bob:37544521377]
	-> [PointerTensor | me:15871848716 -> alice:81706895712]
	*crypto provider: me*

# Step five: Getting the result of the computation
After performing the computation it is time to decrypt the result and convert it back into a float.

In [6]:
result = secret_result.get().float_prec()
result

tensor([2.5000, 3.5000, 4.5000])